# Working with Trajectories

In this example you will see how you can use QMzyme to generate multiple QMzyme Models for different trajectory snapshots. 

Classes used in this example:

- [GenerateModel](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.GenerateModel.html)

- [QM_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#qm-treatment)

- [XTB_Method](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.CalculateModel.html#xtb-treatment)

- [CA_terminal TruncationScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.TruncationSchemes.html#QMzyme.TruncationSchemes.CA_terminal)

- [DistanceCutoff SelectionScheme](https://qmzyme.readthedocs.io/en/latest/API/QMzyme.SelectionSchemes.html#QMzyme.SelectionSchemes.DistanceCutoff)


In [1]:
from QMzyme import GenerateModel
from QMzyme.CalculateModel import QM_Method, XTB_Method
from QMzyme.SelectionSchemes import DistanceCutoff
from QMzyme.data import PQR, DCD

/Users/hrk/anaconda3/envs/qmzyme/lib/python3.11/site-packages/MDAnalysis/topology/TPRParser.py:161: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  import xdrlib


In [2]:
# How many frames?
model = GenerateModel(PQR, DCD)
print("Number of Frames: ", model.universe.trajectory.n_frames)

Number of Frames:  50


/Users/hrk/anaconda3/envs/qmzyme/lib/python3.11/site-packages/MDAnalysis/coordinates/DCD.py:165: DeprecationWarning: DCDReader currently makes independent timesteps by copying self.ts while other readers update self.ts inplace. This behavior will be changed in 3.0 to be the same as other readers. Read more at https://github.com/MDAnalysis/mdanalysis/issues/3889 to learn if this change in behavior might affect you.
  warnings.warn("DCDReader currently makes independent timesteps"
/Users/hrk/git/QMzyme/QMzyme/MDAnalysisWrapper.py:21: UserWarning: Element information was missing from input. MDAnalysis.topology.guessers.guess_types was used to infer element types.
  warnings.warn("Element information was missing from input. MDAnalysis.topology.guessers.guess_types was used to infer element types.", UserWarning)


#### QM-only: Loop over frames to generate QMzymeModels
In this first scenario, you will generate five QM-only models using the DistanceCutoff selection scheme with a cutoff of 3 Angstroms.

In [3]:
%%time

qm_method = QM_Method(
    basis_set='6-31G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

for frame in range(0, 50, 10):
    print('\n====================================')
    print(f'             Frame {frame}')
    print('====================================')
    m = GenerateModel(PQR, DCD, frame=frame)
    m.set_catalytic_center('resid 263')
    m.set_region(selection=DistanceCutoff, cutoff=3)
    m.cutoff_3.rename(f'{m.cutoff_3.name}_frame_{frame}')
    m.truncate_region(region=m.regions[-1])
    c_alpha_atoms = m.regions[-1].get_atoms(attribute='name', value='CA')
    m.regions[-1].set_fixed_atoms(atoms=c_alpha_atoms)
    qm_method.assign_to_region(region=m.regions[-1])
    m.write_input()


             Frame 0

Calculating total charge for QMzymeRegion cutoff_3_frame_0_truncated based on charges read from topology attribute 'charge'...
QMzymeRegion cutoff_3_frame_0_truncated has a total charge of 0.
File /Users/hrk/git/QMzyme-documentation/docs/Examples/QCALC/cutoff_3_frame_0_truncated.inp created.

             Frame 10

Calculating total charge for QMzymeRegion cutoff_3_frame_10_truncated based on charges read from topology attribute 'charge'...
QMzymeRegion cutoff_3_frame_10_truncated has a total charge of 0.
File /Users/hrk/git/QMzyme-documentation/docs/Examples/QCALC/cutoff_3_frame_10_truncated.inp created.

             Frame 20

Calculating total charge for QMzymeRegion cutoff_3_frame_20_truncated based on charges read from topology attribute 'charge'...
QMzymeRegion cutoff_3_frame_20_truncated has a total charge of -1.
File /Users/hrk/git/QMzyme-documentation/docs/Examples/QCALC/cutoff_3_frame_20_truncated.inp created.

             Frame 30

Calculating total c

#### QM/xTB: Loop over frames to generate QMzymeModels
In this second scenario, you will generate five QM/xTB models with a QM region consisting of the ligand, EQU and the catalytic ASP103 residue and a large xTB region containing residues within 8 Angstroms of EQU.

In [4]:
%%time

qm_method = QM_Method(
    basis_set='6-31+G*', 
    functional='wB97X-D3', 
    qm_input='OPT FREQ', 
    program='orca'
)

xtb_method = XTB_Method()

for frame in range(0, 50, 10):
    print('\n====================================')
    print(f'             Frame {frame}')
    print('====================================')
    m = GenerateModel(PQR, DCD, frame=frame)
    m.set_catalytic_center('resid 263')
    m.set_region(name='qm_region', selection='resid 263 or resid 103')
    m.set_region(selection=DistanceCutoff, cutoff=8)
    m.cutoff_8.rename(f'{m.cutoff_8.name}_frame_{frame}')
    m.truncate_region(region=m.regions[-1])
    qm_method.assign_to_region(region=m.qm_region)
    xtb_method.assign_to_region(region=m.regions[-1])
    m.write_input()


             Frame 0

Calculating total charge for QMzymeRegion qm_region based on charges read from topology attribute 'charge'...
QMzymeRegion qm_region has a total charge of -1.

Removing duplicate atoms and recalculating charge for calculation.
The original region cutoff_8_frame_0_truncated can still be accessed in the GenerateModel object.

Calculating total charge for QMzymeRegion  based on charges read from topology attribute 'charge'...
QMzymeRegion  has a total charge of -1.
File /Users/hrk/git/QMzyme-documentation/docs/Examples/QCALC/qm_region_cutoff_8_frame_0_truncated.inp created.

             Frame 10

Calculating total charge for QMzymeRegion qm_region based on charges read from topology attribute 'charge'...
QMzymeRegion qm_region has a total charge of -1.

Removing duplicate atoms and recalculating charge for calculation.
The original region cutoff_8_frame_10_truncated can still be accessed in the GenerateModel object.

Calculating total charge for QMzymeRegion  based